In [1]:
from jarvis.utils.general import gpus
gpus.autoselect()

import glob, numpy as np, pandas as pd, tensorflow as tf, time, random
from tensorflow.keras import Input, Model, layers, optimizers, losses, callbacks, utils
from IPython.display import clear_output, HTML, Javascript, display

import sys  
sys.path.append('/home/jjlou/Jerry/jerry_packages')
from jerry_utils import restart_kernel, load_dataset_v1, load_dataset
import jerry_losses, jerry_metrics 

[ 2024-01-26 10:52:22 ] CUDA_VISIBLE_DEVICES automatically set to: 1           


In [2]:
root = '/home/jjlou/Jerry/wsi-arterio/vessel_detection_and_rough_segmentation/data_test'
#color = glob.glob(f'{root}/train_Color.Basic.Aug_x5_sharded/*/*/*')
#gray = glob.glob(f'{root}/train_Gray.Basic.Aug_sharded/*/*')
external_path = sorted(glob.glob(f'{root}/external_processed/*/*'))
hold_path = glob.glob(f'{root}/hold_out_processed/*/*')

In [3]:
#tf.numpy_function creates output with unknown shape.. so need to reshape it
def reshape(image, mask):
    image = tf.reshape(image, [512,512,3])
    mask = tf.reshape(mask, [512,512])
    return tf.cast(image, 'uint8'), tf.cast(mask, 'uint8')

In [4]:
# random flip via tensorflow layers
def aug(images, mask):
    
    images = tf.cast(images, 'uint8')
    mask = tf.cast(mask, 'uint8') 
    images = tf.reshape(images, [784, 784, 3])
    mask = tf.reshape(mask, [784, 784, 1])
    
    mask = tf.stack([mask, mask, mask], -1)
    mask = tf.reshape(mask, [784, 784, 3])
    
    images_mask = tf.concat([images, mask], -1)  
    images_mask = layers.RandomCrop(512, 512)(images_mask)  
    
    image = images_mask[:,:,:3]
    mask = images_mask[:,:,3]
    mask = tf.expand_dims(mask,axis=2)
    
    return tf.cast(image, 'uint8'), tf.cast(mask, 'uint8') 

In [5]:
def crop(dataset):
    dataset = (
    dataset.map(lambda i,m: aug(i,m), num_parallel_calls=tf.data.AUTOTUNE)
    .map(lambda i,m: reshape(i,m), num_parallel_calls=tf.data.AUTOTUNE)
    .batch(1)
)

In [6]:
def process(filelist, folder):
    for f in filelist:
        shard_name = f.split('/')[-1]
        filename = f.split('/')[-2]
        shard_save = f'{root}/{folder}/{filename}/{shard_name}'
        if not glob.glob(f'{shard_save}/*/*/*.snapshot'):
            shard = tf.data.Dataset.load(f)
            shard = (
                shard.map(lambda i,m: aug(i,m), num_parallel_calls=tf.data.AUTOTUNE)
                .map(lambda i,m: reshape(i,m), num_parallel_calls=tf.data.AUTOTUNE)
                .batch(1)
            )
            shard.save(shard_save)
        else:
            continue

In [7]:
shard_name = external_path[-1].split('/')[-1]
filename = external_path[-1].split('/')[-2]
if not glob.glob((f'{root}/external_cropped/{filename}/{shard_name}/*/*/*.snapshot')):
    process(external_path, 'external_cropped')
    restart_kernel()
    time.sleep(5)

In [8]:
process(hold_path, 'hold_cropped')